In [7]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
#import geopandas as gpd
import folium
from folium.plugins import HeatMap
import re
import sklearn
import numpy as np
from sklearn.neighbors import NearestNeighbors
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 5);
sns.set_style('whitegrid')

In [76]:
by_customer = pd.read_csv('./order_by_customer.csv')
customer = pd.read_csv('./customer_basic.csv')
restaurant = pd.read_csv('./restaurant.csv')

In [51]:
by_customer = by_customer.drop('Unnamed: 0',axis=1)
by_customer


,customer_id,VENDOR
0,95B00Z8,189
1,NBHQJXI,195 676 176 265
2,P0RY7TO,92 33 92 33 92 92 92 92 92 134
3,9PJVBKB,676
4,U59I5S8,113 113 583
...,...,...
9995,EVLNX9L,106
9996,4911852,85 197 86
9997,HCEQCH5,843
9998,21XTYX5,681 681


In [4]:
customer

,Unnamed: 0,customer_id,gender,location_number,location_type,latitude_x,longitude_x,city_id,tag_1,tag_2,...,tag_84,tag_85,tag_86,tag_87,tag_88,tag_89,tag_90,tag_91,grand_total,item_count
0,2295661,F8SFBID,0,0,2,0.03290,-78.60000,1.0,1,0,...,0,0,0,0,0,0,0,0,7.100000,2.00000
1,1891238,Y7H5ARO,0,1,2,0.34770,0.60100,1.0,0,0,...,0,0,0,0,0,0,0,0,16.293828,2.44188
2,3472191,0X5LWPM,3,0,2,1.34900,-78.75000,1.0,1,0,...,0,0,0,0,0,0,0,1,16.293828,2.44188
3,3711332,SPWOZIN,0,0,0,-0.27800,-78.56000,1.0,0,1,...,0,0,0,0,0,0,0,0,16.293828,2.44188
4,5256793,5Z468SS,3,0,0,0.54200,0.53370,1.0,1,0,...,0,0,0,0,0,0,0,1,16.293828,2.44188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1654137,6UOCCK3,0,0,2,-0.10315,0.12354,1.0,1,0,...,0,0,0,0,0,0,0,0,16.293828,2.44188
9996,2704492,P584KU2,3,0,2,0.03270,0.53760,1.0,1,0,...,0,0,0,0,0,0,0,1,16.293828,2.44188
9997,5591171,1DBPPPF,3,0,0,0.11237,0.41650,1.0,1,0,...,0,0,0,0,0,0,0,1,16.293828,2.44188
9998,4042570,GDIDS3P,0,1,0,-0.14560,0.10254,1.0,1,0,...,0,0,0,0,0,0,0,1,21.500000,1.00000


In [78]:
for i in restaurant.keys():
    print(i)

Unnamed: 0
vendor_id
vendor_rating2
favorite_by
rated_by
favorite_percentage
rated_percentage
average_price
average_item_sold
latitude
longitude
vendor_category_id
delivery_charge
prepration_time
discount_percentage
status
verified
rank
vendor_rating
one_click_vendor
city_id
tag_1
tag_2
tag_3
tag_4
tag_5
tag_6
tag_7
tag_8
tag_9
tag_10
tag_11
tag_12
tag_13
tag_14
tag_15
tag_16
tag_17
tag_18
tag_19
tag_20
tag_21
tag_22
tag_23
tag_24
tag_25
tag_26
tag_27
tag_28
tag_29
tag_30
tag_31
tag_32
tag_33
tag_34
tag_35
tag_36
tag_37
tag_38
tag_39
tag_40
tag_41
tag_42
tag_43
tag_44
tag_45
tag_46
tag_47
tag_48
tag_49
tag_50
tag_51
tag_52
tag_53
tag_54
tag_55
tag_56
tag_57
tag_58
tag_59
tag_60
tag_61
tag_62
tag_63
tag_64
tag_65
tag_66
tag_67
tag_68
tag_69
tag_70
tag_71
tag_72
tag_73
tag_74
tag_75
tag_76
tag_77
tag_78
tag_79
tag_80
tag_81
tag_82
tag_83
tag_84
tag_85
tag_86
tag_87
tag_88
tag_89
tag_90
tag_91


In [79]:
restaurant

,Unnamed: 0,vendor_id,vendor_rating2,favorite_by,rated_by,favorite_percentage,rated_percentage,average_price,average_item_sold,latitude,...,tag_82,tag_83,tag_84,tag_85,tag_86,tag_87,tag_88,tag_89,tag_90,tag_91
0,0,4,3.926667,14.958667,17,0.378378,0.113333,3.926667,2.597741,-0.588596,...,0,0,0,0,0,0,0,0,0,1
1,1,13,3.781818,18.618182,4,0.345455,0.072727,3.781818,2.138599,-0.471654,...,0,0,0,0,0,0,0,0,0,0
2,2,20,3.991071,21.300893,17,0.318182,0.151786,3.991071,2.603947,-0.407527,...,0,0,0,0,0,0,0,0,0,1
3,3,23,4.275362,20.766667,8,0.362319,0.115942,4.275362,2.298883,-0.585385,...,0,0,0,0,0,0,0,0,0,0
4,4,28,3.700855,13.302564,21,0.147826,0.179487,3.700855,3.054995,0.480602,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,849,4.870588,11.634118,22,0.035294,0.258824,4.870588,1.964706,-1.588060,...,0,0,0,0,0,0,0,0,0,1
96,96,855,4.862069,11.796552,11,0.000000,0.189655,4.862069,2.086207,2.145206,...,0,0,0,0,0,0,0,0,0,1
97,97,856,4.762712,12.676271,16,0.000000,0.271186,4.762712,2.389831,0.251469,...,0,0,0,0,0,0,0,0,0,1
98,98,858,4.827586,10.189655,14,0.034483,0.241379,4.827586,1.793103,0.019817,...,0,0,0,0,0,0,0,0,0,1


In [24]:
customer_array = customer.to_numpy()[:,2:]

# User Base Recommendation

## Assign weights to each feature

### city_id is supper critical, so we assign large weight. (customer_array[:,5]*=100.) Note that latitude_x and	longitude_x is also important but originally has large variance, so we keep them the same.)
### We want to amplify the customer's preference (customer_array[:,6]*=3.)
### grand_total and	item_count may not be so important, we dilute them (customer_array[:,-2:]/=3.)

In [31]:
customer_array[:,-2:]/=3.
customer_array[:,5]*=100.
customer_array[:,6]*=3.

In [32]:
customer_array

array([[0, 0, 2, ..., 0, 0.7888888888888889, 0.2222222222222222],
       [0, 1, 2, ..., 0, 1.8104253111456055, 0.2713200145997644],
       [3, 0, 2, ..., 1, 1.8104253111456055, 0.2713200145997644],
       ...,
       [3, 0, 0, ..., 1, 1.8104253111456055, 0.2713200145997644],
       [0, 1, 0, ..., 1, 2.388888888888889, 0.1111111111111111],
       [0, 0, 2, ..., 0, 1.8104253111456055, 0.2713200145997644]],
      dtype=object)

In [33]:
customer_array[0]

array([0, 0, 2, 0.0329, -78.6, 10000.0, 9.0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.7888888888888889,
       0.2222222222222222], dtype=object)

## fit KNN model

In [34]:
'''
samples = [[0, 0, 2], [1, 0, 0], [0, 0, 1]]
neigh = NearestNeighbors(2, 0.4)
neigh.fit(samples) 
neigh.kneighbors([[0, 0, 1.3]], 2, return_distance=False)
'''
neigh = NearestNeighbors(2, 0.4)
neigh.fit(customer_array) 

c:\users\anban\anaconda3\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=2, radius=0.4 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


NearestNeighbors(n_neighbors=2, radius=0.4)

## Predict

In [36]:
neigh.kneighbors([[0, 0, 2, 0.0329, -78.6, 10000.0, 9.0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.7888888888888889,
       0.2222222222222222]], 2, return_distance=True)

(array([[0.        , 1.02419884]]), array([[   0, 8291]], dtype=int64))

In [43]:
neigh.kneighbors([[0, 0, 2, 0.0329, -78.6, 10000.0, 9.0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.7888888888888889,
       0.2222222222222222]], 2, return_distance=True)

(array([[1.7320508 , 2.01220855]]), array([[   0, 8291]], dtype=int64))

In [42]:
neigh.kneighbors([[0, 0, 2, 0.0329, -78.6, 10000.0, 9.0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0,
       0.2222222222222222]], 2, return_distance=True)

(array([[5.02802385, 5.02802385]]), array([[ 307, 4973]], dtype=int64))

## Give Recommendation

In [74]:
dist, knn = neigh.kneighbors([[0, 0, 2, 0.0329, -78.6, 10000.0, 9.0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0,
       0.2222222222222222]], 2, return_distance=True)
recommendation = []
recommendation_weight = []
print(dist,knn)
for k,i in enumerate(list(knn)):
    print("generate recommendation for %d'th input..."%k)
    print(by_customer.iloc[i])
    recommended_restaurant = by_customer.iloc[i]['VENDOR'].values
    recommended_restaurant = [s.split(' ') for s in recommended_restaurant]
    confidence = {}
    for j in range(len(recommended_restaurant)):
        for restaurant in recommended_restaurant[j]:
            if restaurant not in confidence:
                confidence[restaurant]=0.
            confidence[restaurant]+=1./dist[0][j]
    p = np.asarray(list(confidence.values()))
    p/=np.sum(p)
    keys = list(confidence.keys())
    recommendation.append(np.random.choice(len(confidence.keys()), 1, p=p))
    print("--Recommendation is restaurant: "+keys[recommendation[-1][0]])

[[5.02802385 5.02802385]] [[ 307 4973]]
generate recommendation for 0'th input...
     customer_id VENDOR
307      5ETOL7J     75
4973     6383RPP    207
--Recommendation is restaurant: 207


# Item Base Recommendation

## Assign weights to each feature

### city_id is supper critical, so we assign large weight. (customer_array[:,5]*=100.) Note that latitude_x and	longitude_x is also important but originally has large variance, so we keep them the same.)
### We want to amplify the customer's preference (customer_array[:,6]*=3.)
### grand_total and	item_count may not be so important, we dilute them (customer_array[:,-2:]/=3.)